In [1]:
# Dependencies
import tweepy
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

import time

plt.style.use('seaborn')

In [2]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


consumer_key = os.getenv("bot_consumer_key")
consumer_secret = os.getenv("bot_consumer_secret")
access_token = os.getenv("bot_access_token")
access_token_secret = os.getenv("bot_access_token_secret")


In [3]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [5]:
def parse_requests(tweet, tweet_dict=dict()):
    tweet_data = []
    tweet_id = tweet["id"]
    tweet_user = tweet["user"]["screen_name"]
    tweet_requests = []
    print(tweet_id)
    for mentions in tweet["entities"]["user_mentions"]:
        if mentions["screen_name"] != "PySentiBot":
            tweet_requests.append(mentions["screen_name"])
    
    tweet_dict = {"id":tweet_id,"user":tweet_user,"analysis_requests":tweet_requests}
    return tweet_dict

In [9]:
def scan_for_requests(since_tweet_id):
    search_handle = "@PySentiBot"

    results = api.search(search_handle,since_id = since_tweet_id)

    if(len(results["statuses"]) > 0):
        tweet_data = []

        for tweet in results["statuses"]:
            tweet_data.append(parse_requests(tweet))
            print(tweet_data)

In [10]:
scan_for_requests(0)

933930946721648641
[{'id': 933930946721648641, 'user': 'Sadhana98905082', 'analysis_requests': ['edsheeran']}]
933930057860599809
[{'id': 933930946721648641, 'user': 'Sadhana98905082', 'analysis_requests': ['edsheeran']}, {'id': 933930057860599809, 'user': 'Sadhana98905082', 'analysis_requests': ['KITKAT']}]
933929308753719296
[{'id': 933930946721648641, 'user': 'Sadhana98905082', 'analysis_requests': ['edsheeran']}, {'id': 933930057860599809, 'user': 'Sadhana98905082', 'analysis_requests': ['KITKAT']}, {'id': 933929308753719296, 'user': 'Sadhana98905082', 'analysis_requests': ['Microsoft']}]
933752497436610560
[{'id': 933930946721648641, 'user': 'Sadhana98905082', 'analysis_requests': ['edsheeran']}, {'id': 933930057860599809, 'user': 'Sadhana98905082', 'analysis_requests': ['KITKAT']}, {'id': 933929308753719296, 'user': 'Sadhana98905082', 'analysis_requests': ['Microsoft']}, {'id': 933752497436610560, 'user': 'Sadhana98905082', 'analysis_requests': ['Forbes', 'CocaCola']}]
9337452536